# 使用 LangChain 构建一个 RAG 应用

## RAG 是什么

RAG 是一种将检索到的文档上下文与大语言模型（LLM）结合起来生成答案的技术。

整个过程主要分为以下几个步骤：

1. 加载文档：将原始数据(来源可能是在线网站、本地文件、各类平台等)加载到 LangChain 中。
1. 文档分割：将加载的文档分割成较小的块，以适应模型的上下文窗口，并更容易进行向量嵌入和检索。
1. 存储嵌入：将分割后的文档内容嵌入到向量空间，并存储到向量数据库中，以便后续检索。
1. 检索文档：通过查询向量数据库，检索与问题最相关的文档片段。
1. 生成回答：将检索到的文档片段与用户问题组合，生成并返回答案。

通过这些步骤，可以构建一个强大的问答系统，将复杂任务分解为更小的步骤并生成详细回答。

![rag](../images/rag.png)

In [2]:
!pip install langchain langchain_community

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
!pip install langchain-chroma -i https://pypi.org/simple

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 3.7 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 33.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.2 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53725 sha256=775d19484f81938711c0b1e43dd1f66da119ab07fb8dc2b39632e758cf23fd68
  Stored in directory: /root/.cache/pip/wheels/e1/26/51/d0bffb3d2fd82256676d7ad3003faea3bd6dddc9577af665f4
Successfully built pypika


## **RAG 开发指南**

**本指南将详细介绍如何使用 LangChain 框架构建一个基于检索增强生成 (RAG) 的应用。**

下面是基于 LangChain 实现的 RAG 的核心步骤与使用到的关键代码抽象（类型、方法、库等）:

1. **加载文档**: 使用 `WebBaseLoader` 类从指定来源加载内容，并生成 `Document` 对象（依赖 `bs4` 库）。
2. **文档分割**: 使用 `RecursiveCharacterTextSplitter` 类的 `split_documents()` 方法将长文档分割成较小的块。
3. **存储嵌入**: 使用 `Chroma` 类的 `from_documents()` 方法将分割后的文档内容嵌入向量空间，并存储在向量数据库中（使用 `OpenAIEmbeddings`），并可以通过检查存储的向量数量来确认存储成功。。
4. **检索文档**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法基于查询从向量数据库中检索最相关的文档片段。
5. **生成回答**: 使用 `ChatOpenAI` 类的 `invoke()` 方法，将检索到的文档片段与用户问题结合，生成回答（通过 `RunnablePassthrough` 和 `StrOutputParser`）。

我们使用的文档是Lilian Weng撰写的《LLM Powered Autonomous Agents》博客文章（https://lilianweng.github.io/posts/2023-06-23-agent/ ），最终构建好的 RAG 应用支持我们询问关于该文章内容的相关问题。

In [3]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Step 1: 加载文档

- **描述**: 使用 `DocumentLoader` 从指定来源（如网页）加载内容，并将其转换为 `Document` 对象。
- **重要代码抽象**:
  - 类: `WebBaseLoader`
  - 方法: `load()`
  - 库: `bs4` (BeautifulSoup)
- **代码解释**:
  - **文档加载**: 使用 `WebBaseLoader` 从网页加载内容，并通过 `BeautifulSoup` 解析 HTML，提取重要的部分。
  - **检查加载数量**: 打印加载的文档数量，确保所有文档正确加载。
  - **验证文档内容**: 输出第一个文档的部分内容，确认加载的数据符合预期。

In [4]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("postTitle","postBody"))
loader = WebBaseLoader(
    web_paths=("https://www.cnblogs.com/JavaEdge/p/18559650",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [5]:
print(docs)

[Document(page_content='\n\nLLM部署，你必须要知道的几个技巧！\n\n\n\n0 前言\n今天我会首先解释为什么 LLM 的部署很难，因为许多人可能并不理解其中的复杂性。接着，我会分享七个提高 LLM 部署效果的技巧和方法。\n1 为啥 LLM 部署困难？\n“最近在忙啥？”\n“我一直在让 LLM 服务变得更简单。”\n“LLM 部署难吗？不是直接调用 OpenAI API 就行？”\n“某种程度上是这样。”因为提到 LLM，大多数人只会想到 OpenAI，调用 API 确实简单。她为什么要谈这些内容？调用 API 谁不会？但实际上，访问 LLM 的方式不止一种。可用托管的API如 OpenAI、Cohere、Anthropic 和 AI21 Labs 等。他们已为你完成托管和部署，你只需调它们。虽然这确实减少你的工作量，但仍存在复杂性，如减少幻觉输出。不过，他们已经完成很多繁重任务。很多场景，你可能更倾向自托管，如调用 Mistral或托管 Llama 或其他模型。这意味着你在自己的环境中托管它，无论VPC还是PC。\n那为啥还自托管？\n很多原因：\n\n降低大规模部署成本。如只做概念验证，基于 OpenAI API 模型成本确实低。但如大规模部署，自托管最终成本更低。因为只需解决自己的业务问题，可用更小模型，而 OpenAI 必须托管一个能解决编程和写作莎士比亚问题的大模型，因此需要更大的模型。大规模部署时，自托管成本会低得多\n性能提升。当你用特定任务的LLM或对其微调，使其专注你的任务，通常得到更好性能\n大多数客户选择自托管的原因：隐私和安全。如你处受监管行业，如需遵循 GDPR 或满足合规团队的要求，你可能也需自托管\n\n如果这几点不重要，就用 API 够了。\n企业选择开源的主要原因\n包括控制权、定制化和成本。最重要的是控制权。拥有 AI 独立性至关重要，如当 OpenAI 再次解雇 CEO，你仍可访问自己的模型，尤其是当你构建重要的业务应用时。如果你正在考虑自托管，你绝对不是孤军奋战，大多数企业都在努力建立自托管能力。\n对冲基金的一员说：“隐私对我的用例很重要，因此自托管是有意义的。”然后他可能会问：“自托管真的有那么难吗？”我经常听到类似的话，这让我非常恼火。答案是：确实更难。你不能忽视那些你看不到的复杂性。

In [6]:
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

8902


In [7]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])



LLM部署，你必须要知道的几个技巧！



0 前言
今天我会首先解释为什么 LLM 的部署很难，因为许多人可能并不理解其中的复杂性。接着，我会分享七个提高 LLM 部署效果的技巧和方法。
1 为


### Step 2: 文档分割

- **描述**: 使用文本分割器将加载的长文档分割成较小的块，以便嵌入和检索。
- **重要代码抽象**:
  - 类: `RecursiveCharacterTextSplitter`
  - 方法: `split_documents()`
- **代码解释**:
  - **文档分割**: 使用 `RecursiveCharacterTextSplitter` 按字符大小分割文档块，设置块大小和重叠字符数，确保文档块适合模型处理。
  - **检查块数量**: 打印分割后的文档块数量，确保分割操作正确执行。
  - **验证块大小**: 输出第一个块的字符数，确认分割块的大小是否符合预期。

In [8]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [9]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

14


In [10]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

699


In [11]:
print(all_splits[0].page_content)  # 打印第一个块的内容

LLM部署，你必须要知道的几个技巧！



0 前言
今天我会首先解释为什么 LLM 的部署很难，因为许多人可能并不理解其中的复杂性。接着，我会分享七个提高 LLM 部署效果的技巧和方法。
1 为啥 LLM 部署困难？
“最近在忙啥？”
“我一直在让 LLM 服务变得更简单。”
“LLM 部署难吗？不是直接调用 OpenAI API 就行？”
“某种程度上是这样。”因为提到 LLM，大多数人只会想到 OpenAI，调用 API 确实简单。她为什么要谈这些内容？调用 API 谁不会？但实际上，访问 LLM 的方式不止一种。可用托管的API如 OpenAI、Cohere、Anthropic 和 AI21 Labs 等。他们已为你完成托管和部署，你只需调它们。虽然这确实减少你的工作量，但仍存在复杂性，如减少幻觉输出。不过，他们已经完成很多繁重任务。很多场景，你可能更倾向自托管，如调用 Mistral或托管 Llama 或其他模型。这意味着你在自己的环境中托管它，无论VPC还是PC。
那为啥还自托管？
很多原因：

降低大规模部署成本。如只做概念验证，基于 OpenAI API 模型成本确实低。但如大规模部署，自托管最终成本更低。因为只需解决自己的业务问题，可用更小模型，而 OpenAI 必须托管一个能解决编程和写作莎士比亚问题的大模型，因此需要更大的模型。大规模部署时，自托管成本会低得多
性能提升。当你用特定任务的LLM或对其微调，使其专注你的任务，通常得到更好性能
大多数客户选择自托管的原因：隐私和安全。如你处受监管行业，如需遵循 GDPR 或满足合规团队的要求，你可能也需自托管


In [12]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'https://www.cnblogs.com/JavaEdge/p/18559650', 'start_index': 2}


### Step 3: 存储嵌入

- **描述**: 将分割后的文档内容嵌入到向量空间中，并存储到向量数据库，以便后续检索。
- **重要代码抽象**:
  - 类: `Chroma`
  - 方法: `from_documents()`
  - 类: `OpenAIEmbeddings`
- **代码解释**:
  - **存储嵌入**: 使用 `Chroma.from_documents()` 方法将所有分割的文档片段进行嵌入(`OpenAIEmbeddings`嵌入模型)，将文档片段嵌入向量空间，并存储在向量数据库中。

#### Chroma 基础使用

**下面是初始化 Chroma 数据库（仅实例化，未存储向量数据）的常见做法：**

**使用构造函数初始化**: 在本地持久化存储 Chroma 数据库.

```python
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)
```

**使用 Cleint 初始化**: 更方便地访问底层数据库/集合。

```python
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)
```


**我们直接使用 `Chroma.from_documents()` 方法 实例化+数据存储**:

该方法返回 Chroma 实例，数据类型为`langchain_chroma.vectorstores.Chroma`，详细 API 文档： https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html

In [13]:
import os


In [62]:
!pip install httpx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceBgeEmbeddings(model_name="/data/model/bge-large-zh-v1.5", model_kwargs=model_kwargs)
                        
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings
)

/root/anaconda3/envs/testyy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# 查看 vectorstore 数据类型
type(vectorstore) 

langchain_chroma.vectorstores.Chroma

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [16]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [17]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [18]:
retrieved_docs = retriever.invoke("企业选择开源的主要原因?")

In [19]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

6


In [20]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

如果这几点不重要，就用 API 够了。
企业选择开源的主要原因
包括控制权、定制化和成本。最重要的是控制权。拥有 AI 独立性至关重要，如当 OpenAI 再次解雇 CEO，你仍可访问自己的模型，尤其是当你构建重要的业务应用时。如果你正在考虑自托管，你绝对不是孤军奋战，大多数企业都在努力建立自托管能力。
对冲基金的一员说：“隐私对我的用例很重要，因此自托管是有意义的。”然后他可能会问：“自托管真的有那么难吗？”我经常听到类似的话，这让我非常恼火。答案是：确实更难。你不能忽视那些你看不到的复杂性。当你调用基于 API 的模型时，你受益于他们的工程师在构建推理和服务基础设施方面所做的所有努力。实际上，像 OpenAI 这样的公司有 50 到 100 人的团队在管理这些基础设施。包括模型压缩、Kubernetes、批处理服务器、函数调用、JSON 生成、运行时引擎等。当你使用 API 模型时，这些你都不需要操心，但当你自托管时，这些问题突然变成了你的责任。
他可能会说：“但我经常部署机器学习模型，比如 XGBoost 或线性回归模型。部署这些 LLM 会有多难？”我们的回答是：“你知道 L 代表什么吗？”部署这些模型要困难得多。为什么呢？LLM 中的第一个 L 代表“大”（Large）。我记得我们刚成立公司时，认为一个拥有 1 亿参数的 BERT 模型已经算大了。现在，一个拥有 70 亿参数的模型被认为是小型模型，但它仍然有 14GB 的大小，这绝对不小。
第二个原因是 GPU。与 CPU 相比，GPU 更难处理，它们也更昂贵，因此高效利用 GPU 十分重要。如果你对 CPU 的利用率不高，可能问题不大，因为它们成本低得多。但对于 GPU，成本、延迟和性能之间的权衡非常明显，这是以前可能没有遇到过的。


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [21]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI()

In [23]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

/root/anaconda3/envs/testyy/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [24]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [25]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [26]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [27]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [28]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 流式生成回答
for chunk in rag_chain.stream("企业选择开源的主要原因??"):
    print(chunk, end="", flush=True)

企业选择开源的主要原因包括控制权、定制化和成本。控制权对于拥有AI独立性至关重要，尤其在构建重要的业务应用时。此外，开源模型的自托管能力让企业可以更灵活地管理基础设施和模型替换周期。

In [31]:
# 流式生成回答
for chunk in rag_chain.stream("什么是LLM部署?"):
    print(chunk, end="", flush=True)

LLM部署是指大型语言模型的部署过程。在部署LLM时，需要考虑部署边界、硬件选择、模型替换周期等因素，以提高性能和效率。选择自托管、考虑GPU利用率优化以及量化模型等技巧可以帮助简化LLM部署过程。

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [32]:
from langchain_core.prompts import PromptTemplate
template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
filler context

用户问：
filler question

请用中文回答用户问题。
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [33]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)


你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
filler context

用户问：
filler question

请用中文回答用户问题。



In [34]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [35]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("什么是LLM部署?")

'我无法回答您的问题。'